Network Architecture 
<img src=https://camo.githubusercontent.com/f2ca62a4157c3afd0e4426c8c473ed1a17f0bcd1/68747470733a2f2f756361726563646e2e636f6d2f38343333376436332d663261612d346639312d623564622d6465316436373136346632372f>

In [4]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

In [5]:
# Weights for the Dense Layers and pre-attention LSTM should remainsame for all T's
Tx=30
repeator = RepeatVector(30) 
densor1  = Dense(10,activation='tanh')
densor2 = Dense(1,activation ='relu')
cocatenator = Concatenante(axis=-1)
activator = Activation('softmax',name='attention-weights')
dotor = Dot(axes=1)

In [7]:
def one_step_attention(a,s_prev):
    
    """
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s) n_s=n_s
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    print(alphas.shape)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = np.sum(dotor([alphas,a]))
    print(context.shape)
    
    return context


In [8]:
#Again some globally defined entities
#For this case:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"
post_attentions_LSTM_cell = LSTM(n_s,return_state=True)
ouput_layer = Dense(10,activation='softmax')

In [9]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, input_vocab_size, output_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "input_vocab"
    machine_vocab_size -- size of the python dictionary "output_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, input_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
        
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    a = Bidirectional(LSTM(n_a,return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a,s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs=context,initial_state=[s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs=[X,s0,c0] ,outputs=outputs)
    
    return model

In [ ]:
#Just incase if you are stuck 
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
opt = Adam(lr=0.005,beta_1=0.9,beta_2=0.999,decay=0.01)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)